In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
traffic_seoul = pd.read_csv('../Data/newSeoul_2005_2019.csv')
traffic_seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [3]:
traffic_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      4500 non-null   int64 
 1   월       4500 non-null   int64 
 2   자치구명    4500 non-null   object
 3   발생건수    4500 non-null   int64 
 4   사망자수    4500 non-null   int64 
 5   부상자수    4500 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 211.1+ KB


In [5]:
traffic_seoul_2019 = traffic_seoul[traffic_seoul['년도'] == 2019]
traffic_seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158


In [6]:
# index 정리하기
traffic_seoul_2019.reset_index(drop=True,inplace=True)
traffic_seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [10]:
traffic_anal=traffic_seoul_2019.pivot_table(index='자치구명',values=['발생건수','부상자수','사망자수'],aggfunc=sum)
traffic_anal.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [13]:
seoul_limit = pd.read_csv('../Data/seoul.csv',encoding='euc-kr')
seoul_limit.head()

,area,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [14]:
seoul_limit.columns = ['자치구명','경도','위도']

In [15]:
data_result = pd.merge(traffic_anal,seoul_limit,on='자치구명')
data_result.head()

,자치구명,발생건수,부상자수,사망자수,경도,위도
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [20]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   경도      25 non-null     float64
 5   위도      25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


In [19]:
# folium은 int를 모른다. float로 바꿔줘야 된다. 
data_result['발생건수'] = data_result['발생건수'].astype(float)
data_result['부상자수'] = data_result['부상자수'].astype(float)
data_result['사망자수'] = data_result['사망자수'].astype(float)

In [21]:
# 자치구의 구성을 표시하기
import folium

In [43]:
seoul_map=folium.Map(
     [37.55,126.98],
    zoom_start = 12
)

for lng , lat, name , count ,death in \
zip(data_result['위도'],data_result['경도'],data_result['자치구명'], data_result['발생건수'],data_result['사망자수']):
    folium.Marker(
        [lng,lat],
        popup=name
    ).add_to(seoul_map)

    folium.CircleMarker(
        [lng,lat],
        radius=count/50, # 요걸 크기에 따라서 바꿔줄 수 있다.
        color='blue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.7
    ).add_to(seoul_map)

    folium.CircleMarker(
        [lng,lat],
        radius=death, # 요걸 크기에 따라서 바꿔줄 수 있다.
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7
    ).add_to(seoul_map)
    

seoul_map